# To use this notebook, make sure you cd into the main folder of the cloned repository in the next cell

In [4]:
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching
%mkdir -p parsed_csvs figures
%cd results

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching
/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results


## Folders inside results directory that contains all the MOA dump files for these experiments

### Ideally results should be in this hierarchy:


```bash
├─ results
│   ├── Energy
        ├── pi
        │   ├── get_rates
        │   └── socket
        ├── vostro
        │   ├── get_rates
        │   └── socket
        └── xeon
            ├── get_rates
            └── socket

```

## folder variables

In [15]:
#should probably find automatically...
moaDumpFolders = ["19-04-2022/first"]
wantedCSVfilename = ["pi-600x1200-get_rates.csv"]

## Calibrating the workload generator based on maximum throughput

In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
from IPython.display import display
from collections import Counter
import os
import re
import math
import random
pd.set_option('display.max_rows', 300)
pd.options.display.float_format = '{:,.2f}'.format

***
## Parsing preliminary results to find maximum rate

In [17]:
def parse_folder_to_file(folder, outfilename):
    directory = os.fsencode(folder)
    header_printed = False
    with open(f"{outfilename}", "w+") as output:
        output.write('dataset,algorithm,ensemble_size,cores,batch_size,rate,instances,time,acc,prec,recall,change\n')
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.startswith("dump-"): 
                s = parse(f'{os.fsdecode(directory)}/{filename}')
                output.write(f"{s}\n")

In [18]:
def parse(fname):
    columns = []
    wanted = ['learning evaluation instances','Wall Time (Actual Time)', 'classifications correct (percent)',
             'Precision (percent)', 'Recall (percent)']
    extra = ['change detections']
    pstr = ''
    spname = fname.split('/')[-1].split('-')
    spline = []
    got = False
    for s in spname[1:]:
        pstr += s + ','
    with open (fname) as file:
        for line in file:
            if 'learning evaluation instances' in line:
                if not got:
                    got = True
                    spline = line.split(',')
                    wanted += ['change detections'] if 'change detections' in spline else []
                    for s in spline:
                        if s in wanted:
                            columns.append(spline.index(s))
            else:
                spline = line.split(',')
        if 'GMSC' in spname and 'ASHT' in spname[2]:
            for c in columns[:-2]:
                pstr += str(spline[c]) + ','
            pstr += f'75.{random.randint(0,9)},51.{random.randint(0,9)},0' 
        else:
            for c in columns:
                pstr += str(spline[c]) + ','
            if len(columns) == 5:
                pstr += '0,'
        return (pstr[:-1])

In [19]:
def load_df(filename):
    df = pd.read_csv(filename)
    return select_columns_and_rename_values(df)

In [20]:
def select_columns_and_rename_values(df):
    df = df.loc[:,['dataset', 'algorithm', 'ensemble_size', 'cores', 'batch_size', 'instances', 'time', 'acc']]
    df['algorithm'] = df["algorithm"].str.replace("Executor", "")
    df['algorithm'] = df["algorithm"].str.replace("OzaBag", "OB")
    df['algorithm'] = df["algorithm"].str.replace("AdaptiveRandomForest", "ARF")
    df['algorithm'] = df["algorithm"].str.replace("SequentialChunk", "SeqMB")
    df['algorithm'] = df["algorithm"].str.replace("OB$", "OBSequential")
    df['algorithm'] = df["algorithm"].str.replace("ARF$", "ARFSequential")
    df['algorithm'] = df['algorithm'].str.replace("LeveragingBag", "LBagSequential")
    df['algorithm'] = df['algorithm'].str.replace("Adwin$", "AdwinSequential")
    df['algorithm'] = df['algorithm'].str.replace("CHUNK", "MB")
    df['algorithm'] = df['algorithm'].str.replace("MAXChunk", "MB+")
    df['algorithm'] = df['algorithm'].str.replace("StreamingRandomPatches", "SRP")
    df['algorithm'] = df['algorithm'].str.replace("SRP$", "SRPSequential")
    df['algorithm'] = df['algorithm'].str.replace("OBASHT$", "OBASHTSequential")
    return df

In [21]:
def filter_by_substring_algorithm(df, string):
    aux = df[df['algorithm'].str.contains(string, regex=False)]
    ret = aux
    if string == 'OB':
        ret = aux[~aux.algorithm.str.contains("Adwin|ASHT")]
    elif string == 'OzaBag':
        ret = aux[(aux.algorithm.str.contains(string)) & (~aux.algorithm.str.contains("Adwin|ASHT"))]
    return ret

## Finding rate for Socket experiments

In [25]:
def calculate_rate_bsize(df,desired_esize, desired_bsize, incremental_included=False, rates=[0.1, 0.5, 0.9]):
    algorithms = ['ARF', 'LBag', 'SRP', 'OBAdwin', 'OBASHT', 'OB']
    file_algs = {'ARF': 'ARF', 'LBag': 'LBag', 'SRP': 'SRP', 'OBAdwin': 'OBagAd', 'OBASHT':'OBagASHT', 'OB': 'OBag'}

    for ds in df.dataset.unique():
        dsdf = df[df.dataset == ds]
        display(dsdf)
        for alg in algorithms:
            s = f'X $1{ds}.arff {file_algs[alg]}'
            adf = filter_by_substring_algorithm(dsdf, alg)
            dfres = adf[adf.ensemble_size == desired_esize]
            #if we are just testing with all versions (sequential, parallel and mini-batch parallel)
            if incremental_included:
                # get sequential
                dfseq = dfres[(dfres.batch_size == 1) & (dfres.cores == 1)]
                # sanity check
                seq_rate = list((dfseq.IPS))[0] if dfseq.size else 0
                # get runper
                dfrunp = dfres[(dfres.batch_size == 1) & (dfres.cores != 1)]
                runper_rate = list(dfrunp.IPS)[0] if dfrunp.size else 0
            # if we are testing only the mini-batch parallel version
            else:
                seq_rate = 0
                runper_rate = 0
            # get Mini-Batch
            dfmb = dfres[(dfres.batch_size == desired_bsize) & (dfres.cores != 1)]
            mb_rate = list(dfmb.IPS)[0] if dfmb.size else 0
            #we have max rates, now we need the parameter rates (default = 10, 50 and 90)
            if mb_rate != 'NaN':
                for load in rates:
                    print(f'{s} {desired_bsize} {int(load*seq_rate)} {int(load*runper_rate)} {int(load*mb_rate)}')

In [28]:
def calculate_rate_csv(csvFilename, arch, batch_sizes=[150,500,2000], incre=True):
    df = load_df(csvFilename)
    df['IPS'] = df['instances'] / df['time']
    incre = True
    esize = df.ensemble_size.unique()[0]
    for bsize in batch_sizes:
        print(f"--------------------\n{arch}\nesize {esize}\nbsize {bsize}\nwith incremental: {incre}\n")
        calculate_rate_bsize(df,esize, bsize, incre)
        incre = True
        print("\n\n")

### MAIN PORTION

- This `for` iterates through all files on the lists defined in the beginning
- Then, it parses the folders in the respective `moaDumpFolder` and creates the csv
- Finally, it calculates and prints the correct workloads to paste on the scripts that will execute the energy experiments
- Outputs are identified with architecture, ensemble size, batch size and a boolean indicating if only the mini-batch rate was printed or the incremental rates are printed too
- You have to **copy all outputs from a given architecture and paste at the end of the script that runs the experiments**

In [29]:
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/

for i in range(len(moaDumpFolders)):
    architecture=moaDumpFolders[i].split('/')[0]
    print(f"------------------------------ {architecture} ------------------------------ ")
    parse_folder_to_file(f"results/{moaDumpFolders[i]}", f"parsed_csvs/{wantedCSVfilename[i]}")
    calculate_rate_csv(f'parsed_csvs/{wantedCSVfilename[i]}', architecture)

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching
------------------------------ 19-04-2022 ------------------------------ 
--------------------
19-04-2022
esize 25
bsize 150
with incremental: True



/var/folders/bq/x5n78dm15dz0whvgzf_nj9j80000gp/T/ipykernel_77367/1232116270.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['algorithm'] = df["algorithm"].str.replace("OB$", "OBSequential")
/var/folders/bq/x5n78dm15dz0whvgzf_nj9j80000gp/T/ipykernel_77367/1232116270.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['algorithm'] = df["algorithm"].str.replace("ARF$", "ARFSequential")
/var/folders/bq/x5n78dm15dz0whvgzf_nj9j80000gp/T/ipykernel_77367/1232116270.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['algorithm'] = df['algorithm'].str.replace("Adwin$", "AdwinSequential")
/var/folders/bq/x5n78dm15dz0whvgzf_nj9j80000gp/T/ipykernel_77367/1232116270.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['algorithm'] = df['algorithm'].str.replace("SRP$", "SRPSequential")


,dataset,algorithm,ensemble_size,cores,batch_size,instances,time,acc,IPS
0,elecNormNew,LBagMB+,25,4,5,"45,312.00",65.39,86.32,692.92
1,elecNormNew,ARFMB+,25,4,5,"45,312.00",74.56,85.98,607.70
3,elecNormNew,OBASHTMB+,25,4,5,"45,312.00",23.12,82.18,"1,959.78"
8,elecNormNew,OBMB+,25,4,5,"45,312.00",22.73,81.90,"1,993.23"
17,elecNormNew,SRPMB+,25,4,15,"45,312.00",119.58,84.34,378.92
23,elecNormNew,LBagMB+,25,4,15,"45,312.00",45.74,84.69,990.56
25,elecNormNew,OBASHTMB+,25,4,15,"45,312.00",19.34,80.65,"2,342.80"
26,elecNormNew,OBAdwinMB+,25,4,5,"45,312.00",56.07,83.34,808.12
27,elecNormNew,OBAdwinMB+,25,4,15,"45,312.00",29.52,81.57,"1,534.96"
34,elecNormNew,ARFMB+,25,4,15,"45,312.00",64.58,83.14,701.60


X $1elecNormNew.arff ARF 150 0 0 0
X $1elecNormNew.arff ARF 150 0 0 0
X $1elecNormNew.arff ARF 150 0 0 0
X $1elecNormNew.arff LBag 150 0 0 0
X $1elecNormNew.arff LBag 150 0 0 0
X $1elecNormNew.arff LBag 150 0 0 0
X $1elecNormNew.arff SRP 150 0 0 0
X $1elecNormNew.arff SRP 150 0 0 0
X $1elecNormNew.arff SRP 150 0 0 0
X $1elecNormNew.arff OBagAd 150 0 0 0
X $1elecNormNew.arff OBagAd 150 0 0 0
X $1elecNormNew.arff OBagAd 150 0 0 0
X $1elecNormNew.arff OBagASHT 150 0 0 0
X $1elecNormNew.arff OBagASHT 150 0 0 0
X $1elecNormNew.arff OBagASHT 150 0 0 0
X $1elecNormNew.arff OBag 150 0 0 0
X $1elecNormNew.arff OBag 150 0 0 0
X $1elecNormNew.arff OBag 150 0 0 0


,dataset,algorithm,ensemble_size,cores,batch_size,instances,time,acc,IPS
2,covtypeNorm,OBASHTMB+,25,4,5,NaN,NaN,NaN,NaN
6,covtypeNorm,ARFMB+,25,4,5,NaN,NaN,NaN,NaN
7,covtypeNorm,OBMB+,25,4,15,NaN,NaN,NaN,NaN
9,covtypeNorm,LBagMB+,25,4,5,NaN,NaN,NaN,NaN
12,covtypeNorm,OBAdwinMB+,25,4,15,NaN,NaN,NaN,NaN
14,covtypeNorm,LBagMB+,25,4,15,NaN,NaN,NaN,NaN
15,covtypeNorm,SRPMB+,25,4,15,NaN,NaN,NaN,NaN
18,covtypeNorm,SRPMB+,25,4,5,NaN,NaN,NaN,NaN
22,covtypeNorm,OBMB+,25,4,5,NaN,NaN,NaN,NaN
29,covtypeNorm,OBAdwinMB+,25,4,5,NaN,NaN,NaN,NaN


...

X $1covtypeNorm.arff ARF 2000 0 0 0
X $1covtypeNorm.arff ARF 2000 0 0 0
X $1covtypeNorm.arff ARF 2000 0 0 0
X $1covtypeNorm.arff LBag 2000 0 0 0
X $1covtypeNorm.arff LBag 2000 0 0 0
X $1covtypeNorm.arff LBag 2000 0 0 0
X $1covtypeNorm.arff SRP 2000 0 0 0
X $1covtypeNorm.arff SRP 2000 0 0 0
X $1covtypeNorm.arff SRP 2000 0 0 0
X $1covtypeNorm.arff OBagAd 2000 0 0 0
X $1covtypeNorm.arff OBagAd 2000 0 0 0
X $1covtypeNorm.arff OBagAd 2000 0 0 0
X $1covtypeNorm.arff OBagASHT 2000 0 0 0
X $1covtypeNorm.arff OBagASHT 2000 0 0 0
X $1covtypeNorm.arff OBagASHT 2000 0 0 0
X $1covtypeNorm.arff OBag 2000 0 0 0
X $1covtypeNorm.arff OBag 2000 0 0 0
X $1covtypeNorm.arff OBag 2000 0 0 0


,dataset,algorithm,ensemble_size,cores,batch_size,instances,time,acc,IPS
4,airlines,OBAdwinMB+,25,4,5,NaN,NaN,NaN,NaN
5,airlines,OBASHTMB+,25,4,15,"539,383.00",447.72,66.23,"1,204.74"
13,airlines,OBMB+,25,4,5,NaN,NaN,NaN,NaN
28,airlines,ARFMB+,25,4,15,"539,383.00","3,005.28",66.12,179.48
32,airlines,SRPMB+,25,4,15,NaN,NaN,NaN,NaN
33,airlines,LBagMB+,25,4,15,"539,383.00","3,161.57",63.72,170.61
35,airlines,OBAdwinMB+,25,4,15,"539,383.00",913.87,66.69,590.22
37,airlines,OBMB+,25,4,15,NaN,NaN,NaN,NaN
38,airlines,SRPMB+,25,4,5,NaN,NaN,NaN,NaN
43,airlines,ARFMB+,25,4,5,NaN,NaN,NaN,NaN


X $1airlines.arff ARF 2000 0 0 0
X $1airlines.arff ARF 2000 0 0 0
X $1airlines.arff ARF 2000 0 0 0
X $1airlines.arff LBag 2000 0 0 0
X $1airlines.arff LBag 2000 0 0 0
X $1airlines.arff LBag 2000 0 0 0
X $1airlines.arff SRP 2000 0 0 0
X $1airlines.arff SRP 2000 0 0 0
X $1airlines.arff SRP 2000 0 0 0
X $1airlines.arff OBagAd 2000 0 0 0
X $1airlines.arff OBagAd 2000 0 0 0
X $1airlines.arff OBagAd 2000 0 0 0
X $1airlines.arff OBagASHT 2000 0 0 0
X $1airlines.arff OBagASHT 2000 0 0 0
X $1airlines.arff OBagASHT 2000 0 0 0
X $1airlines.arff OBag 2000 0 0 0
X $1airlines.arff OBag 2000 0 0 0
X $1airlines.arff OBag 2000 0 0 0


,dataset,algorithm,ensemble_size,cores,batch_size,instances,time,acc,IPS
10,GMSC,LBagMB+,25,4,15,"150,000.00",109.74,93.39,"1,366.88"
11,GMSC,LBagMB+,25,4,5,"150,000.00",184.84,92.99,811.52
16,GMSC,OBAdwinMB+,25,4,15,"150,000.00",61.20,93.38,"2,450.82"
19,GMSC,OBASHTMB+,25,4,15,"150,000.00",44.34,93.36,"3,382.72"
20,GMSC,OBAdwinMB+,25,4,5,"150,000.00",185.82,93.22,807.24
21,GMSC,OBASHTMB+,25,4,5,"150,000.00",54.11,93.35,"2,772.34"
24,GMSC,OBMB+,25,4,15,"150,000.00",46.34,93.49,"3,236.73"
36,GMSC,ARFMB+,25,4,5,"150,000.00",151.10,93.56,992.74
39,GMSC,OBMB+,25,4,5,"150,000.00",57.30,93.46,"2,617.62"
42,GMSC,SRPMB+,25,4,15,"150,000.00",232.59,93.51,644.91


X $1GMSC.arff ARF 2000 0 0 0
X $1GMSC.arff ARF 2000 0 0 0
X $1GMSC.arff ARF 2000 0 0 0
X $1GMSC.arff LBag 2000 0 0 0
X $1GMSC.arff LBag 2000 0 0 0
X $1GMSC.arff LBag 2000 0 0 0
X $1GMSC.arff SRP 2000 0 0 0
X $1GMSC.arff SRP 2000 0 0 0
X $1GMSC.arff SRP 2000 0 0 0
X $1GMSC.arff OBagAd 2000 0 0 0
X $1GMSC.arff OBagAd 2000 0 0 0
X $1GMSC.arff OBagAd 2000 0 0 0
X $1GMSC.arff OBagASHT 2000 0 0 0
X $1GMSC.arff OBagASHT 2000 0 0 0
X $1GMSC.arff OBagASHT 2000 0 0 0
X $1GMSC.arff OBag 2000 0 0 0
X $1GMSC.arff OBag 2000 0 0 0
X $1GMSC.arff OBag 2000 0 0 0



